In [5]:
import math
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

Positional Encoding

In [6]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, \
        MAX_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        pos = torch.arrange(MAX_len).unsqueeze(1)
        div_term = torch.exp(torch.arrange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(MAX_len, 1, d_model)
        
        # sin for even num indices
        pe[:, 0, 0::2] = torch.sin(pos * div_term)
        # cos for odd num indices
        pe[:, 0, 1::2] = torch.cos(pos * div_term)

        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        x = x + self.pe[:x.size(0)]
        # Args: x.shape = [seq_len, batch_size, embedding_dim]
        return self.dropout(x)

Modeling

In [8]:
class TransformerModel(nn.Module):
    
    def __init__(self, ntoken: int, d_model:int, nhead: int, d_hid: int,\
        nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead,\
            d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        src = self.encoder(src) * math.sqrt(self.d_model)
        
        


SyntaxError: positional argument follows keyword argument (3623080884.py, line 9)